In [5]:
import plotly.express as px
import numpy as np
import pandas as pd

numpisos = 4
nzcol = 0

xcol = []
ycol = []
zcol = []

for i in range(numpisos):
    numcolj = 4
    numcoli = 3
    nycol = 0
    nxcol = 0
    for j in range(numcolj):
        nxcol = 0
        for d in range(numcoli):
            xcol.append(nxcol)
            ycol.append(nycol)
            zcol.append(nzcol)
            nxcol += 6
        nycol += 4
    nzcol += 3

# Create a DataFrame
df_nodes = pd.DataFrame({'x': xcol, 'y': ycol, 'z': zcol})

coordinates = np.zeros((numpisos*numcoli*numcolj, 3))
for i in range(len(xcol)):
    # Define elements by specifying the indices of connected nodes
    coordinates[i,:] = [xcol[i], ycol[i], zcol[i]]

tbl_coordinates = pd.DataFrame(coordinates)
# display(tbl_coordinates)


# # Create a DataFrame for elements
# df_elements = pd.DataFrame(elements, columns=['node1', 'node2'])

# # Create an interactive 3D scatter plot for nodes
# fig = px.scatter_3d(df_nodes, x='x', y='y', z='z', title='Interactive 3D Building Plot')

# # Create traces for the elements (lines connecting nodes)
# for element in elements:
#     node1 = element[0]
#     node2 = element[1]
#     trace = go.Scatter3d(
#         x=[df_nodes.loc[node1, 'x'], df_nodes.loc[node2, 'x']],
#         y=[df_nodes.loc[node1, 'y'], df_nodes.loc[node2, 'y']],
#         z=[df_nodes.loc[node1, 'z'], df_nodes.loc[node2, 'z']],
#         mode='lines',
#         line=dict(color='red', width=3),  # Customize line color and width
#         name=f'Element {node1}-{node2}'
#     )
#     fig.add_trace(trace)

# # Customize the plot if needed
# fig.update_layout(scene=dict(xaxis_title='X-axis Label', yaxis_title='Y-axis Label', zaxis_title='Z-axis Label'))

# # Show the interactive plot
# fig.show()



In [3]:
# Create an interactive 3D scatter plot
fig = px.scatter_3d(df_nodes, x='x', y='y', z='z', title='Interactive 3D Scatter Plot')

# Customize the plot if needed
fig.update_layout(scene_aspectmode='manual', scene_aspectratio=dict(x=1, y=1, z=1))


# Show the interactive plot
fig.show()


In [4]:
'''
OpenSees -- Open System for Earthquake Engineering Simulation
Pacific Earthquake Engineering Research Center
http://opensees.berkeley.edu/
Basic Truss Example 1.1
-----------------------
 2d 3 Element Elastic Truss
 Single Nodal Load, Static Analysis

Example Objectives
------------------
 Simple Introduction to OpenSees

Units: kips, in, sec
Written: Andreas Schellenberg (andreas.schellenberg@gmail.com)
Date: June 2017
'''

# import the OpenSees Python module
import openseespy.opensees as op
import vfo.vfo as vfo


# ------------------------------
# Start of model generation
# ------------------------------

# remove existing model
op.wipe()

# create ModelBuilder (with two-dimensions and 2 DOF/node)
op.model("BasicBuilder", "-ndm",2, "-ndf",2)

# create nodes & add to Domain - command: node nodeId xCrd yCrd
op.node(1, 0.0,    0.0)
op.node(2, 144.0,  0.0)
op.node(3, 168.0,  0.0)
op.node(4,  72.0, 96.0)

# set the boundary conditions - command: fix nodeID xRestrnt? yRestrnt?
op.fix(1, 1, 1)
op.fix(2, 1, 1)
op.fix(3, 1, 1)


# Define materials for truss elements
# -----------------------------------
# Create Elastic material prototype - command: uniaxialMaterial Elastic matID E
op.uniaxialMaterial("Elastic", 1, 3000.0)

# Define elements
# ---------------
# Create truss elements - command: element truss trussID node1 node2 A matID
op.element("truss", 1, 1, 4, 10.0, 1)
op.element("truss", 2, 2, 4,  5.0, 1)
op.element("truss", 3, 3, 4,  5.0, 1)

# Define loads
# ------------
# create a Linear TimeSeries (load factor varies linearly with time) - command: timeSeries Linear $tag
op.timeSeries("Linear", 1)

# create a Plain load pattern - command: pattern Plain $tag $timeSeriesTag { $loads }
op.pattern("Plain", 1, 1, "-fact", 1.0)
# create the nodal load - command: load nodeID xForce yForce
op.load(4, 100.0, -50.0)


vfo.createODB(model="First_openseespy_truss")

# render the model after defining all the nodes and elements
vfo.plot_model()

# plot mode shape
vfo.plot_model(model="First_openseespy_truss",show_nodetags="yes")



>>> VFO Warning: No 'loadcase' argument was provided to save output.<<<
No Model_ODB specified, trying to get data from the active model.


Widget(value="<iframe src='http://localhost:51240/index.html?ui=P_0x24a98c9b610_0&reconnect=auto' style='width…

A view with name (P_0x24a98c9b610_0) is already registered
 => returning previous one


Widget(value="<iframe src='http://localhost:51240/index.html?ui=P_0x24a98c9b610_0&reconnect=auto' style='width…

Reading data from the First_openseespy_truss_ODB.


Widget(value="<iframe src='http://localhost:51240/index.html?ui=P_0x24a98c9ba00_1&reconnect=auto' style='width…

A view with name (P_0x24a98c9ba00_1) is already registered
 => returning previous one


Widget(value="<iframe src='http://localhost:51240/index.html?ui=P_0x24a98c9ba00_1&reconnect=auto' style='width…

In [ ]:
# print model
#printModel()
op.printModel("-JSON", "-file", "Example1.1.json")


# ------------------------------
# End of model generation
# ------------------------------

In [ ]:
# ------------------------------
# Start of analysis generation
# ------------------------------

# create the system of equation, a SPD using a band storage scheme
op.system("BandSPD")

# create the DOF numberer, the reverse Cuthill-McKee algorithm
op.numberer("RCM")

# create the constraint handler, a Plain handler is used as homo constraints
op.constraints("Plain")

# create the solution algorithm, a Linear algorithm is created
op.algorithm("Linear")

# create the integration scheme, the LoadControl scheme using steps of 1.0
op.integrator("LoadControl", 1.0)

# create the analysis object 
op.analysis("Static")

# ------------------------------
# End of analysis generation
# ------------------------------


# ------------------------------
# Start of recorder generation
# ------------------------------

# create a Recorder object for the nodal displacements at node 4
op.recorder("Node", "-file", "example.out", "-time", "-node", 4, "-dof", 1, 2, "disp")

# create a recorder for element forces, one in global and the other local system
op.recorder("Element", "-file", "eleGlobal.out", "-time", "-ele", 1, 2, 3, "forces")
op.recorder("Element", "-file", "eleLocal.out", "-time", "-ele", 1, 2, 3, "basicForces")

# ------------------------------
# End of recorder generation
# ------------------------------


# ------------------------------
# Finally perform the analysis
# ------------------------------

# perform the analysis
op.analyze(1)


# ------------------------------
# Print Stuff to Screen
# ------------------------------

# print the current state at node 4 and at all elements
#print("node 4 displacement: ", nodeDisp(4))
op.printModel("node", 4)
op.printModel("ele")
op.wipe()


 Node: 4
	Coordinates  : 72 96 
	Disps: 0.530093 -0.177894 
	Velocities   : 0 0 
	 unbalanced Load: 100 -50 
	ID : 0 1 

Element: 1 type: Truss  iNode: 1 jNode: 4 Area: 10 Mass/Length: 0 cMass: 0 
	 strain: 0.00146451 axial load: 43.9352 
	 unbalanced load: -26.3611 -35.1482 26.3611 35.1482 
 	 Material: 
Element: 2 type: Truss  iNode: 2 jNode: 4 Area: 5 Mass/Length: 0 cMass: 0 
	 strain: -0.00383642 axial load: -57.5463 
	 unbalanced load: -34.5278 46.0371 34.5278 -46.0371 
 	 Material: 
Element: 3 type: Truss  iNode: 3 jNode: 4 Area: 5 Mass/Length: 0 cMass: 0 
	 strain: -0.00368743 axial load: -55.3114 
	 unbalanced load: -39.1111 39.1111 39.1111 -39.1111 
 	 Material: 
